In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
from datetime import datetime, date, timedelta
#--- Web scriping ---
import requests
from bs4 import BeautifulSoup
#--- ------------ ---
#from googletrans import Translator 

# <font style="font-size: 80px"><font color="yellow">C</font><font style="color: whitesmoke">onsume </font><font color="yellow">P</font><font style="color: whitesmoke">roduct </font><font color="yellow">I</font><font style="color: whitesmoke">ndex </font>(<font color="yellow">CPI</font>)</font>

### Getting data

In [2]:
url = 'https://thedocs.worldbank.org/en/doc/1ad246272dbbc437c74323719506aa0c-0350012021/original/Inflation-data.xlsx'
local_path = 'From_WorldData.xlsx'

try: 
    response = requests.get(url)

    if(response.status_code == 200):
        with open(local_path, 'wb') as file:
            file.write(response.content)
        print(f'Download success, status code {response.status_code}')
    else:
        print(f'Download Error, status code {response.status_code}')
except requests.exceptions.RequestException as e:
    print(f'Error: {e}') 
    
    pass

Download success, status code 200


In [3]:
excel = "From_WorldData.xlsx"
sheets = openpyxl.load_workbook(excel) #https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG

In [4]:
print(sheets.sheetnames)

['Intro', 'top', 'hcpi_m', 'hcpi_q', 'hcpi_a', 'ecpi_m', 'ecpi_q', 'ecpi_a', 'fcpi_m', 'fcpi_q', 'fcpi_a', 'ccpi_m', 'ccpi_q', 'ccpi_a', 'ppi_m', 'ppi_q', 'ppi_a', 'def_q', 'def_a', 'Aggregate']


In [5]:
CPIa = pd.read_excel(excel, sheet_name="hcpi_a")

---

### <font style="color: yellow">Columns</font>

In [6]:
CPIa.drop(columns=["Note", "IMF Country Code", "Indicator Type", "Series Name"], inplace=True)
CPIa.rename(columns={"Country Code":"Code"}, inplace=True)
CPIa[["Code", "Country"]] = CPIa[["Country", "Code"]]
CPIa.rename(columns={"Code":"Country", "Country":"Code"}, inplace=True) #If the name not exist previously to the function are not changed

In [7]:
CPIa.columns = CPIa.columns.astype(str)

In [8]:
len(CPIa["Country"])

205

In [9]:
countryCount = {}
for i in CPIa["Country"]:
    if countryCount.get(i):
        countryCount[i] += 1
    else:
        countryCount[i] = 1

country_repeat = {k: v for k, v in countryCount.items() if(v>=2)}
print(country_repeat)

{nan: 2}


In [10]:
CPIa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 57 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  203 non-null    object 
 1   Code     204 non-null    object 
 2   1970     155 non-null    float64
 3   1971     155 non-null    float64
 4   1972     155 non-null    float64
 5   1973     156 non-null    float64
 6   1974     156 non-null    float64
 7   1975     154 non-null    float64
 8   1976     154 non-null    float64
 9   1977     154 non-null    float64
 10  1978     155 non-null    float64
 11  1979     155 non-null    float64
 12  1980     158 non-null    float64
 13  1981     162 non-null    float64
 14  1982     163 non-null    float64
 15  1983     163 non-null    float64
 16  1984     163 non-null    float64
 17  1985     163 non-null    float64
 18  1986     164 non-null    float64
 19  1987     165 non-null    float64
 20  1988     165 non-null    float64
 21  1989     165 non

---

### <font style="color: yellow"> This data-frame have something wrong elements? </font>

In [11]:
nanCountry = CPIa[CPIa["Country"].isna() == True]
CPIa.drop(index=nanCountry.index, inplace=True)
CPIa.reset_index(inplace=True, drop=True)

In [12]:
#CPIa.loc[CPIa["Country"] == "Angola", "1970"] = "Hello"
aux = CPIa.iloc[:, 2:]
dataFilter = aux.select_dtypes(exclude=["float"]).any()
dataFilter

Series([], dtype: bool)

---

### <font style="color: yellow"> Sorting the data </font>

In [13]:
nanPorcent = {}
noData = []
for i in CPIa["Country"]:
    rowSelected = CPIa[CPIa["Country"] == i]
    if(rowSelected.isna().any().any()):
        nanPorcent[i] = rowSelected.iloc[:, 2:].isna().any().value_counts(normalize=True)[True]
        if(nanPorcent[i] == 1):
            noData.append(int(rowSelected.index.values[0]))
    else:
        nanPorcent[i] = 0.0

print(nanPorcent, "\n", noData)

#--- Del data ---#
for i in range(0, len(noData)):
    del nanPorcent[CPIa.iloc[noData[i]]["Country"]]
CPIa.drop(index=noData, inplace=True)
CPIa.reset_index(inplace=True, drop=True)

{'Aruba': 0.4727272727272727, 'Afghanistan': 0.0, 'Angola': 0.0, 'Albania': 0.36363636363636365, 'United Arab Emirates': 0.0, 'Argentina': 0.0, 'Armenia': 0.41818181818181815, 'Antigua and Barbuda': 0.0, 'Australia': 0.0, 'Austria': 0.0, 'Azerbaijan': 0.38181818181818183, 'Burundi': 0.0, 'Belgium': 0.0, 'Benin': 0.0, 'Burkina Faso': 0.0, 'Bangladesh': 0.0, 'Bulgaria': 0.0, 'Bahrain': 0.0, 'Bahamas, The': 0.0, 'Bosnia and Herzegovina': 0.45454545454545453, 'Belarus': 0.0, 'Belize': 0.0, 'Bolivia': 0.0, 'Brazil': 0.0, 'Barbados': 0.0, 'Brunei Darussalam': 0.0, 'Bhutan': 0.05454545454545454, 'Botswana': 0.0, 'Central African Republic': 0.0, 'Canada': 0.0, 'Switzerland': 0.0, 'Chile': 0.0, 'China': 0.0, "Côte d'Ivoire": 0.0, 'Cameroon': 0.0, 'Congo, Dem. Rep.': 0.0, 'Congo, Rep.': 0.0, 'Colombia': 0.0, 'Comoros': 0.0, 'Cabo Verde': 0.0, 'Costa Rica': 0.0, 'Curacao': 0.12727272727272726, 'Cayman Islands': 0.4727272727272727, 'Cyprus': 0.0, 'Czech Republic': 0.0, 'Germany': 0.0, 'Djibouti': 

In [14]:
nanPorcent = dict(sorted(nanPorcent.items(), key=lambda items: items[1], reverse=False))
newOrder = [x for x in nanPorcent]
aux_df = pd.DataFrame(columns=CPIa.columns)

for i in newOrder:
    rowSelected = CPIa[CPIa["Country"] == i]
    aux_df = pd.concat([aux_df, rowSelected], ignore_index=True)

aux_df = aux_df.round(2)
CPIa =  aux_df
CPIa

C:\Users\matia\AppData\Local\Temp\ipykernel_11260\1748395857.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aux_df = pd.concat([aux_df, rowSelected], ignore_index=True)


,Country,Code,1970,1971,1972,1973,1974,1975,1976,1977,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Afghanistan,AFG,25.51,25.51,-12.52,-10.68,10.23,11.85,0.60,7.65,...,-0.66,4.38,4.98,0.63,2.30,5.44,5.06,10.60,-7.71,-6.60
1,Angola,AGO,7.97,5.78,15.80,15.67,27.42,29.00,80.70,69.01,...,9.16,32.38,29.84,19.63,17.08,21.02,23.85,23.83,13.64,28.24
2,United Arab Emirates,ARE,21.98,21.98,21.98,21.98,21.98,21.98,21.98,21.97,...,4.07,1.62,1.97,3.06,-1.93,-2.08,0.18,4.83,1.62,1.66
3,Argentina,ARG,13.59,34.73,58.45,61.25,23.47,182.93,443.97,176.00,...,26.54,39.40,25.68,34.28,53.55,42.02,48.41,72.43,133.49,219.88
4,Antigua and Barbuda,ATG,8.87,8.63,12.67,21.72,12.97,8.61,15.62,13.75,...,0.97,-0.49,2.42,1.21,1.48,1.06,1.63,7.53,5.07,6.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,Turks and Caicos Islands,TCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.20,2.00,2.10,2.10,2.20,2.30,NaN,NaN,NaN,NaN
198,Liechtenstein,LIE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.11,-0.41,0.51,0.92,0.40,-0.70,0.51,8.40,3.94,2.26
199,South Sudan,SSD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,52.81,379.85,187.87,83.49,51.19,29.68,30.23,-6.69,39.67,128.40
200,British Virgin Islands,VGB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.80,1.10,1.20,2.10,1.40,0.40,2.80,NaN,2.58,NaN


---

In [15]:
CPIa[CPIa["Country"] == "United States"]

,Country,Code,1970,1971,1972,1973,1974,1975,1976,1977,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
142,United States,USA,5.9,4.29,3.27,6.18,11.05,9.14,5.74,6.5,...,0.12,1.26,2.14,2.44,1.81,1.23,4.7,8.0,4.13,2.95


<font style="color: green; font-size: 35px">Save changes</font>

In [16]:
print(CPIa.columns)
CPIa.to_csv("Inflation_clean.csv", index=False)

Index(['Country', 'Code', '1970', '1971', '1972', '1973', '1974', '1975',
       '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984',
       '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993',
       '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020',
       '2021', '2022', '2023', '2024'],
      dtype='object')
